# Packages

In [1]:
# Visualisation
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns
from pyvis.network import Network

# Data analysis / Data processing
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth', None)
pd.options.display.float_format = "{:,.2f}".format
from datetime import time, timedelta, datetime
import numpy as np
import networkx as nx
from collections import defaultdict
import ast

# Maths & Stats
import math 
import scipy.stats as st
from scipy import stats
from scipy.stats import norm
import statsmodels.stats.weightstats as ws
from statsmodels.stats.proportion import test_proportions_2indep
import AB_library
# from ambrosia.designer import Designer
# from ambrosia.tester import Tester
import expab
from statsmodels.stats.power import NormalIndPower
from statsmodels.stats.proportion import proportion_effectsize
from math import ceil

# System library
import os
import ipywidgets
import warnings
warnings.filterwarnings('ignore')
from tqdm.notebook import tqdm
tqdm.pandas()
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
%config InlineBackend.figure_format='retina'
# from itables import init_notebook_mode
# init_notebook_mode(all_interactive=True)
import openpyxl

# Data connection
from google.cloud import bigquery
bigquery_client = bigquery.Client(project='analytics-dev-333113')


# Useful functions
def read_bq(query, project='analytics-dev-333113'):
    client = bigquery.Client(project=project)
    query_job = client.query(query)
    result_df = query_job.to_dataframe()
    return result_df

def display_side_by_side(*args):

    html_str = ''
    for df in args:
        html_str += df.to_html()
    display_html(
        html_str.replace('table','table style="display:inline"'), 
        raw=True
    )

def cycle_sql(start, end, query, weeks=False):
    """
    You have to use {date} in your script to add cycle date into this backets
    """
    date_start = datetime.strptime(start, '%Y-%m-%d')
    date_end = datetime.strptime(end, '%Y-%m-%d')

    if weeks == False:
        daterange = [(date_start + timedelta(days=x)).strftime('%Y-%m-%d') for x in range(((date_end-date_start).days)+1)]
    else:
        daterange = [(date_start + timedelta(weeks=x)).strftime('%Y-%m-%d') for x in range(((date_end-date_start).days//7)+1)] # weeks dividing days by 7

    total_df = pd.DataFrame()

    counter = 0

    for date in daterange:
        counter+=1
        print(f"{counter}) Uploading - {date}:", datetime.today().strftime('%Y-%m-%d %H:%M:%S'))
        script = query.format(date = date)
        df_cycle = bigquery_client.query(script).to_dataframe()
        if df_cycle.empty == True:
            print('Dataframe is empty')
        total_df = pd.concat([df_cycle, total_df])
    return total_df  

def writing_excel(name:str, dataset1=None, dataset2=None, dataset3=None, dataset4=None):
    with pd.ExcelWriter(f"{name}.xlsx") as writer:

    # use to_excel function and specify the sheet_name and index 
    # to store the dataframe in specified sheet

        if dataset1 is not None:
            if dataset2 is not None:
                if dataset3 is not None:
                    if dataset4 is not None:
                        dataset1.to_excel(writer, sheet_name=f"1-{name}", 
                                        #   index=False
                                            )
                        dataset2.to_excel(writer, sheet_name=f"2-{name}", 
                                        #   index=False
                                            )
                        dataset3.to_excel(writer, sheet_name=f"3-{name}", 
                                        #   index=False
                                            )
                        dataset4.to_excel(writer, sheet_name=f"4-{name}", 
                                        #   index=False
                                            )
                    else:
                        dataset1.to_excel(writer, sheet_name=f"1-{name}", 
                                        #   index=False
                                            )
                        dataset2.to_excel(writer, sheet_name=f"2-{name}", 
                                        #   index=False
                                            )
                        dataset3.to_excel(writer, sheet_name=f"3-{name}", 
                                        #   index=False
                                            )
                else:
                    dataset1.to_excel(writer, sheet_name=f"1-{name}", 
                                    #   index=False
                                        )
                    dataset2.to_excel(writer, sheet_name=f"2-{name}", 
                                    #   index=False
                                        )
            else:
                dataset1.to_excel(writer, sheet_name=f"1-{name}", 
                                #   index=False
                                    )

        print('DataFrame is written to Excel File successfully.') 

# Design

In [2]:
df_sample_size = read_bq("""
SELECT user_id,
       os_name,
       city_id,
       city_name,
       country_id,
       country_name,
       COALESCE(filled_flow, 'liveness')                                  AS filled_flow,
       COALESCE(MAX(IF(name = 'client.verification_start.show', client_time, NULL)),
                MAX(IF((name = 'client.verification_flow_result_status.show' AND
                        LOWER(status) = 'approve'), client_time, NULL)),
                MAX(IF((name = 'client.verification_flow_result_status.show' AND
                        LOWER(status) != 'approve'), client_time, NULL))) AS show_dt,
       COALESCE(MAX(IF(name = 'client.verification_start.click', client_time, NULL)),
                MAX(IF((name = 'client.verification_flow_result_status.show' AND
                        LOWER(status) = 'approve'), client_time, NULL)),
                MAX(IF((name = 'client.verification_flow_result_status.show' AND
                        LOWER(status) != 'approve'), client_time, NULL))) AS click_dt,
       MAX(IF((name = 'client.verification_flow_result_status.show' AND
               LOWER(status) = 'approve'), client_time, NULL))            AS approve_dt,
       MAX(IF((name = 'client.verification_flow_result_status.show' AND
               LOWER(status) != 'approve'), client_time, NULL))           AS not_approve_dt
FROM (SELECT t1.user_id,
             t1.name,
             t1.os_name,
             DATE(TIMESTAMP_MILLIS(t1.client_time))                  AS event_dt_part,
             TIMESTAMP_MILLIS(t1.client_time)                        AS client_time,
             t1.city_id,
             t2.city_name,
             t2.country_id,
             t2.country_name,
             JSON_EXTRACT_SCALAR(payload, '$.verification_flow')     AS verification_flow,
             IF(JSON_EXTRACT_SCALAR(payload, '$.verification_flow') IS NULL,
                LAG(JSON_EXTRACT_SCALAR(payload, '$.verification_flow'))
                    OVER (PARTITION BY t1.user_id ORDER BY client_time),
                JSON_EXTRACT_SCALAR(payload, '$.verification_flow')) AS filled_flow,
             JSON_EXTRACT_SCALAR(payload, '$.status')                AS status
      FROM (SELECT *
            FROM indriver-e6e40.emart.product_event t1
            WHERE 1 = 1
              AND name IN (
                           'client.verification_start.show',
                           'client.verification_start.click',
                           'client.verification_flow_result_status.show'
                )
              AND event_dt_part BETWEEN '2025-06-01' AND CURRENT_DATE()
              AND city_id IN (4269)
            QUALIFY
                ROW_NUMBER() OVER (PARTITION BY user_id, name, os_name, event_dt_part, JSON_EXTRACT_SCALAR(payload, '$.verification_flow') ORDER BY client_time DESC) =
                1) t1
               JOIN indriver-e6e40.heap.vw_macroregion_mapping t2
                    ON
                        t1.city_id = t2.city_id
      WHERE 1 = 1
        AND name IN (
                     'client.verification_start.show', 'client.verification_start.click',
                     'client.verification_flow_result_status.show'
          ))
GROUP BY 1, 2, 3, 4, 5, 6, 7
""")

df_sample_size.head()

,user_id,os_name,city_id,city_name,country_id,country_name,filled_flow,show_dt,click_dt,approve_dt,not_approve_dt
0,38875844,android,4269,Puerto Montt,25,Chile,liveness,2025-06-03 18:00:08.882000+00:00,2025-06-03 18:00:10.568000+00:00,2025-06-03 18:00:54.631000+00:00,NaT
1,48661658,android,4269,Puerto Montt,25,Chile,liveness,2025-06-01 22:22:46.516000+00:00,2025-06-01 22:22:47.469000+00:00,2025-06-01 22:23:14.537000+00:00,NaT
2,113640930,ios,4269,Puerto Montt,25,Chile,liveness,2025-06-09 21:29:37.896000+00:00,2025-06-09 21:29:40.246000+00:00,2025-06-09 21:30:18.852000+00:00,NaT
3,152172509,android,4269,Puerto Montt,25,Chile,liveness,2025-06-15 21:29:29.134000+00:00,2025-06-15 21:29:31.005000+00:00,NaT,2025-06-15 21:29:10.194000+00:00
4,177572555,android,4269,Puerto Montt,25,Chile,liveness,2025-06-12 18:11:44.214000+00:00,2025-06-12 18:11:44.952000+00:00,2025-06-12 18:12:16.282000+00:00,NaT


In [25]:
df_sample_size['show_dt'] = pd.to_datetime(df_sample_size['show_dt'], errors='coerce')
df_sample_size['show_dt'] = df_sample_size['show_dt'].dt.date
df_sample_size['show_dt'] = pd.to_datetime(df_sample_size['show_dt'], errors='coerce')
df_sample_size['is_approved'] = df_sample_size['approve_dt'].notna().astype(int)


In [29]:
df_sample_size.query("show_dt>'2025-06-01'").groupby(['show_dt'], as_index=False)['user_id'].nunique()['user_id'].mean()*6*0.4

1345.327659574468

In [35]:
def calculate_mde_unequal_split(
    baseline_conversion_rate: float,
    alpha: float = 0.05,
    power: float = 0.8,
    n_total: int = None,
    allocation_ratio: tuple = (0.5, 0.5) # Соотношение (контроль, тест), например (0.7, 0.3)
) -> dict:
    """
    Рассчитывает MDE (минимально обнаруживаемый эффект) для A/B-теста
    с неравномерным распределением групп.

    Args:
        baseline_conversion_rate (float): Базовая конверсия контрольной группы (от 0 до 1).
        alpha (float): Уровень значимости (ошибка I рода), по умолчанию 0.05.
        power (float): Статистическая мощность (1 - ошибка II рода), по умолчанию 0.8.
        n_total (int): Общий размер выборки для обеих групп.
                       Если не указан, функция вернет ошибку, так как MDE зависит от размера выборки.
        allocation_ratio (tuple): Кортеж, представляющий соотношение распределения
                                  (доля контрольной группы, доля тестовой группы).
                                  Сумма долей должна быть равна 1.0.
                                  По умолчанию (0.5, 0.5) - равномерное распределение.

    Returns:
        dict: Словарь, содержащий рассчитанный MDE (абсолютный и относительный),
              а также размеры групп и другие параметры.
              Возвращает None, если n_total не указан.
    """
    if n_total is None:
        return {"error": "Для расчета MDE необходимо указать общий размер выборки (n_total)."}

    if not (0 < baseline_conversion_rate < 1):
        raise ValueError("Базовая конверсия должна быть между 0 и 1.")
    if not (0 < alpha < 1):
        raise ValueError("Уровень значимости (альфа) должен быть между 0 и 1.")
    if not (0 < power < 1):
        raise ValueError("Мощность должна быть между 0 и 1.")
    if not (np.isclose(sum(allocation_ratio), 1.0) and all(r > 0 for r in allocation_ratio)):
        raise ValueError("Соотношение распределения должно быть кортежем положительных чисел, сумма которых равна 1.0.")

    # Разделяем общий размер выборки согласно соотношению
    n_control = ceil(n_total * allocation_ratio[0])
    n_variant = ceil(n_total * allocation_ratio[1])

    # Убедимся, что n_control + n_variant не превышает n_total (из-за округления ceil)
    # и корректируем, если это так
    if n_control + n_variant > n_total:
        if allocation_ratio[0] > allocation_ratio[1]:
            n_control = n_total - n_variant
        else:
            n_variant = n_total - n_control
    
    # Если одна из групп слишком мала после округления, убедимся, что она не 0
    if n_control == 0:
        n_control = 1
        n_variant = n_total - 1
    if n_variant == 0:
        n_variant = 1
        n_control = n_total - 1
    
    # Рассчитываем отношение размеров групп для NormalIndPower
    # NormalIndPower ожидает nobs1 (размер первой группы) и ratio (nobs2 / nobs1)
    # Мы используем n_control как nobs1, а n_variant как nobs2
    ratio_nobs = n_variant / n_control if n_control > 0 else 1 # Избегаем деления на ноль

    # Создаем объект для расчета мощности
    power_calculator = NormalIndPower()

    # Находим размер эффекта (Cohen's h)
    # solve_power возвращает effect_size, если остальные параметры заданы
    effect_size_cohen_h = power_calculator.solve_power(
        effect_size=None,
        nobs1=n_control,
        alpha=alpha,
        power=power,
        ratio=ratio_nobs,
        alternative='two-sided' # Двусторонний тест (обнаруживаем как увеличение, так и уменьшение)
    )

    # Преобразуем Cohen's h обратно в конверсию тестовой группы
    # effect_size = 2 * arcsin(sqrt(p2)) - 2 * arcsin(sqrt(p1))
    # Мы знаем p1 (baseline_conversion_rate) и effect_size_cohen_h
    # Нужно найти p2
    arcsin_sqrt_p1 = np.arcsin(np.sqrt(baseline_conversion_rate))
    arcsin_sqrt_p2 = arcsin_sqrt_p1 + (effect_size_cohen_h / 2) # Для увеличения
    
    # Конверсия не может быть больше 1
    detectable_conversion_rate_variant = np.sin(arcsin_sqrt_p2)**2
    if detectable_conversion_rate_variant > 1:
        detectable_conversion_rate_variant = 1.0

    # MDE (абсолютная разница)
    mde_absolute = detectable_conversion_rate_variant - baseline_conversion_rate

    # MDE (относительная разница, в процентах)
    mde_relative_percentage = (mde_absolute / baseline_conversion_rate) * 100 if baseline_conversion_rate != 0 else float('inf')

    return {
        "baseline_conversion_rate": baseline_conversion_rate,
        "alpha": alpha,
        "power": power,
        "n_total": n_total,
        "n_control": int(n_control),
        "n_variant": int(n_variant),
        "allocation_ratio": allocation_ratio,
        "effect_size_cohen_h": effect_size_cohen_h,
        "detectable_conversion_rate_variant": detectable_conversion_rate_variant,
        "mde_absolute": mde_absolute,
        "mde_relative_percentage": mde_relative_percentage
    }

def analyze_mde_scenarios(
    baseline_conversion_rate: float,
    alpha_levels: list[float],
    power_levels: list[float],
    n_total_levels: list[int],
    allocation_ratio: tuple = (0.5, 0.5)
) -> pd.DataFrame:
    """
    Анализирует MDE для различных сценариев, возвращая результаты в DataFrame.

    Args:
        baseline_conversion_rate (float): Базовая конверсия контрольной группы (от 0 до 1).
        alpha_levels (list[float]): Список уровней значимости (альфа) для тестирования.
        power_levels (list[float]): Список уровней статистической мощности для тестирования.
        n_total_levels (list[int]): Список общих размеров выборок для тестирования.
        allocation_ratio (tuple): Кортеж, представляющий соотношение распределения
                                  (доля контрольной группы, доля тестовой группы).

    Returns:
        pd.DataFrame: DataFrame, содержащий MDE и размеры групп для каждого сценария.
    """
    results = []
    for n_total in n_total_levels:
        for alpha in alpha_levels:
            for power in power_levels:
                beta = 1 - power # Ошибка второго рода
                
                mde_result = calculate_mde_unequal_split(
                    baseline_conversion_rate=baseline_conversion_rate,
                    alpha=alpha,
                    power=power,
                    n_total=n_total,
                    allocation_ratio=allocation_ratio
                )
                
                if "error" not in mde_result:
                    results.append({
                        "Alpha": alpha,
                        "Beta": beta,
                        "N_Total": n_total,
                        "Allocation_Ratio": allocation_ratio,
                        "N_Control": mde_result["n_control"],
                        "N_Variant": mde_result["n_variant"],
                        "MDE_Absolute": mde_result["mde_absolute"],
                        "MDE_Relative_Percentage": mde_result["mde_relative_percentage"]
                    })
                else:
                    print(f"Ошибка для N_Total={n_total}, Alpha={alpha}, Power={power}: {mde_result['error']}")

    df_results = pd.DataFrame(results)
    
    # Добавляем столбец с форматированной строкой для ошибок
    df_results['Errors (α, β)'] = df_results.apply(
        lambda row: f"({row['Alpha']}; {row['Beta']:.1f})", axis=1
    )
    
    # Добавляем столбец с форматированной строкой для размеров групп
    df_results['Group sizes'] = df_results.apply(
        lambda row: f"Control: {row['N_Control']}; Test: {row['N_Variant']}", axis=1
    )

    # Переупорядочиваем столбцы для соответствия запрошенному формату
    # 'Metric' не является прямым столбцом, это скорее категория.
    # Я представлю MDE_Absolute и MDE_Relative_Percentage как отдельные столбцы.
    final_columns = [
        "Errors (α, β)",
        "N_Total",
        "Allocation_Ratio",
        "Group sizes",
        "MDE_Absolute",
        "MDE_Relative_Percentage"
    ]
    
    return df_results[final_columns]


weekly = df_sample_size.query("show_dt>'2025-06-01'").groupby(['show_dt'], as_index=False)['user_id'].nunique()['user_id'].mean()*6*0.4
baseline_cr = 0.7156002875629044
alpha_levels_to_test = [0.01, 0.05]
power_levels_to_test = [0.8] 
n_total_levels_to_test = [int(weekly), int(weekly)*2, int(weekly*4)]
allocation_ratio_to_test = (0.9, 0.1) 

mde_analysis_df = analyze_mde_scenarios(
    baseline_conversion_rate=baseline_cr,
    alpha_levels=alpha_levels_to_test,
    power_levels=power_levels_to_test,
    n_total_levels=n_total_levels_to_test,
    allocation_ratio=allocation_ratio_to_test
)

mde_analysis_df

,"Errors (α, β)",N_Total,Allocation_Ratio,Group sizes,MDE_Absolute,MDE_Relative_Percentage
0,(0.01; 0.2),1345,"(0.9, 0.1)",Control: 1210; Test: 135,0.13,17.80
1,(0.05; 0.2),1345,"(0.9, 0.1)",Control: 1210; Test: 135,0.11,14.89
2,(0.01; 0.2),2690,"(0.9, 0.1)",Control: 2421; Test: 269,0.09,13.01
3,(0.05; 0.2),2690,"(0.9, 0.1)",Control: 2421; Test: 269,0.08,10.80
4,(0.01; 0.2),5381,"(0.9, 0.1)",Control: 4842; Test: 539,0.07,9.38
5,(0.05; 0.2),5381,"(0.9, 0.1)",Control: 4842; Test: 539,0.06,7.75


# Monitoring

In [16]:
df = read_bq("""
WITH newbies AS (SELECT user_id,
                        metric_date
                 FROM indriver-bi.incity.tbl_incity_growth_metrics_detail
                 WHERE user_type = 'pass'
                   AND rides_count > 0
                   AND metric_date >= DATE_ADD(CURRENT_DATE(), INTERVAL -1 YEAR)
                   AND country_id = 25),
     gmv AS (SELECT user_id,
                    SUM(gmv_clean_usd) AS gmv,
                    SUM(rides_count)   AS rides,
                    SUM(orders_count)  AS orders
             FROM indriver-bi.incity.tbl_incity_growth_metrics_detail
             WHERE user_type = 'pass'
               AND metric_date BETWEEN '2025-06-01' AND CURRENT_DATE()
               AND country_id = 25
             GROUP BY 1),
     total AS (SELECT t1.user_id,
                      t1.city_id,
                      geo.city_name,
                      geo.country_id,
                      geo.country_name,
                      IF(group_id = 4546528, 0, 1) AS group_id,
                      participant_first_toggle_date,
                      t2.metric_date,
                      CASE
                          WHEN t2.metric_date IS NULL THEN 1
                          ELSE 0
                          END                         newbie_flag
               FROM indrive-core.ab_platform.tbl_ab_experiment_markup t1
                        JOIN indriver-e6e40.heap.vw_macroregion_mapping geo
                             ON
                                 t1.city_id = geo.city_id
                        LEFT JOIN newbies t2
                                  ON t1.user_id = t2.user_id AND t2.metric_date < t1.participant_first_toggle_date
               WHERE experiment_id = 3713
               QUALIFY ROW_NUMBER() OVER (PARTITION BY t1.user_id ORDER BY metric_date DESC) = 1),
     liveness AS (SELECT user_id,
                         os_name,
                         city_id,
                         city_name,
                         country_id,
                         country_name,
                         COALESCE(filled_flow, 'liveness')                                  AS filled_flow,
                         COALESCE(MAX(IF(name = 'client.verification_start.show', client_time, NULL)),
                                  MAX(IF((name = 'client.verification_flow_result_status.show' AND
                                          LOWER(status) = 'approve'), client_time, NULL)),
                                  MAX(IF((name = 'client.verification_flow_result_status.show' AND
                                          LOWER(status) != 'approve'), client_time, NULL))) AS show_dt,
                         COALESCE(MAX(IF(name = 'client.verification_start.click', client_time, NULL)),
                                  MAX(IF((name = 'client.verification_flow_result_status.show' AND
                                          LOWER(status) = 'approve'), client_time, NULL)),
                                  MAX(IF((name = 'client.verification_flow_result_status.show' AND
                                          LOWER(status) != 'approve'), client_time, NULL))) AS click_dt,
                         MAX(IF((name = 'client.verification_flow_result_status.show' AND
                                 LOWER(status) = 'approve'), client_time, NULL))            AS approve_dt,
                         MAX(IF((name = 'client.verification_flow_result_status.show' AND
                                 LOWER(status) != 'approve'), client_time, NULL))           AS not_approve_dt
                  FROM (SELECT t1.user_id,
                               t1.name,
                               t1.os_name,
                               DATE(TIMESTAMP_MILLIS(t1.client_time))                  AS event_dt_part,
                               TIMESTAMP_MILLIS(t1.client_time)                        AS client_time,
                               t1.city_id,
                               t2.city_name,
                               t2.country_id,
                               t2.country_name,
                               JSON_EXTRACT_SCALAR(payload, '$.verification_flow')     AS verification_flow,
                               IF(JSON_EXTRACT_SCALAR(payload, '$.verification_flow') IS NULL,
                                  LAG(JSON_EXTRACT_SCALAR(payload, '$.verification_flow'))
                                      OVER (PARTITION BY t1.user_id ORDER BY client_time),
                                  JSON_EXTRACT_SCALAR(payload, '$.verification_flow')) AS filled_flow,
                               JSON_EXTRACT_SCALAR(payload, '$.status')                AS status
                        FROM (SELECT *
                              FROM indriver-e6e40.emart.product_event t1
                              WHERE 1 = 1
                                AND name IN (
                                             'client.verification_start.show',
                                             'client.verification_start.click',
                                             'client.verification_flow_result_status.show'
                                  )
                                AND event_dt_part BETWEEN '2025-06-01' AND CURRENT_DATE()
                                AND country_id IN (25)
                              QUALIFY
                                  ROW_NUMBER() OVER (PARTITION BY user_id, name, os_name, event_dt_part, JSON_EXTRACT_SCALAR(payload, '$.verification_flow') ORDER BY client_time DESC) =
                                  1) t1
                                 JOIN indriver-e6e40.heap.vw_macroregion_mapping t2
                                      ON
                                          t1.city_id = t2.city_id
                        WHERE 1 = 1
                          AND name IN (
                                       'client.verification_start.show', 'client.verification_start.click',
                                       'client.verification_flow_result_status.show'
                            ))
                  GROUP BY 1, 2, 3, 4, 5, 6, 7),
     total_liv AS (SELECT t1.user_id,
                          t1.city_id,
                          t1.city_name,
                          t1.country_id,
                          t1.country_name,
                          t1.group_id,
                          t1.participant_first_toggle_date,
                          t1.newbie_flag,
                          t2.user_id AS user_with_svf,
                          t2.os_name,
                          t2.filled_flow,
                          t2.show_dt,
                          t2.click_dt,
                          t2.approve_dt,
                          t2.not_approve_dt
                   FROM total t1
                            JOIN liveness t2
                                 ON t1.user_id = t2.user_id AND DATE(t2.show_dt) >= t1.participant_first_toggle_date),
     rides AS (SELECT order_uuid,
                      user_id    AS pass_id,
                      driver_id,
                      city_id    AS order_city_id,
                      country_id AS order_country_id,
                      status_order,
                      order_timestamp,
                      at_pickup_dttm,
                      departed_pickup_dttm,
                      at_destination_dttm,
                      departed_destination_dttm,
                      driveraccept_timestamp,
                      driverarrived_timestamp,
                      driverstarttheride_timestamp,
                      driverdone_timestamp,
                      clientdone_timestamp,
                      clientcancel_timestamp,
                      drivercancel_timestamp,
                      user_reg_date,
                      driver_reg_date,
                      stage,
                      created_date_order_part,
                      duration_in_seconds
               FROM indriver-e6e40.imart.incity_detail_new_order
               WHERE created_date_order_part BETWEEN '2025-06-01'
                   AND CURRENT_DATE()
                 AND status_order = 'RIDE_STATUS_DONE'
                 AND driveraccept_timestamp IS NOT NULL
                 AND (clientcancel_timestamp IS NULL
                   AND drivercancel_timestamp IS NULL))
SELECT t1.user_id,
       t1.group_id,
       t1.participant_first_toggle_date,
       t1.os_name,
       t1.city_id,
       t1.city_name,
       t1.country_id,
       t1.country_name,
       t1.newbie_flag,
       filled_flow,
       show_dt,
       click_dt,
       approve_dt                                                  AS approve_dt,
       IF(approve_dt IS NOT NULL, 1, 0)                            AS approve_flag,
       not_approve_dt,
       IF(not_approve_dt IS NOT NULL AND approve_dt IS NULL, 1, 0) AS not_approve_flag,
       t2.order_timestamp,
       IF(t2.order_timestamp IS NOT NULL, 1, 0)                    AS order_flag,
       t3.gmv,
       t3.rides,
       t3.orders
FROM total_liv t1
         LEFT JOIN rides t2
                   ON t1.user_id = t2.pass_id AND
                      t2.created_date_order_part >= participant_first_toggle_date
         LEFT JOIN gmv t3 ON t1.user_id = t3.user_id
QUALIFY ROW_NUMBER() OVER (PARTITION BY t1.user_id ORDER BY t2.order_timestamp) = 1
""")

df.head()

,user_id,group_id,participant_first_toggle_date,os_name,city_id,city_name,country_id,country_name,newbie_flag,filled_flow,show_dt,click_dt,approve_dt,approve_flag,not_approve_dt,not_approve_flag,order_timestamp,order_flag,gmv,rides,orders
0,21921294,0,2025-08-07,android,4269,Puerto Montt,25,Chile,0,liveness,2025-09-06 22:17:33.475000+00:00,2025-09-06 22:17:34.708000+00:00,2025-09-06 22:18:35.560000+00:00,1,NaT,0,2025-08-07 03:46:05+00:00,1,86.16,36,57
1,62136015,1,2025-08-07,android,4269,Puerto Montt,25,Chile,0,liveness,2025-09-06 02:04:06.580000+00:00,2025-09-06 02:04:13.094000+00:00,2025-09-06 02:04:43.664000+00:00,1,NaT,0,2025-08-07 02:29:09+00:00,1,102.15,35,60
2,75149470,0,2025-08-18,ios,4269,Puerto Montt,25,Chile,0,liveness,2025-08-18 19:24:53.882000+00:00,2025-08-18 19:24:54.925000+00:00,2025-08-18 19:25:28.427000+00:00,1,NaT,0,2025-08-20 03:54:37+00:00,1,129.69,45,52
3,118829587,1,2025-08-12,android,4269,Puerto Montt,25,Chile,0,documents,2025-08-12 01:34:40.320000+00:00,2025-08-12 01:34:46.376000+00:00,2025-08-12 01:35:32.171000+00:00,1,NaT,0,2025-08-12 01:35:37+00:00,1,69.83,24,49
4,166519297,1,2025-08-07,android,4269,Puerto Montt,25,Chile,0,liveness,2025-08-29 19:55:10.758000+00:00,2025-08-29 19:55:19.852000+00:00,2025-08-29 19:55:52.289000+00:00,1,NaT,0,2025-08-11 10:53:20+00:00,1,65.61,17,26


In [17]:
df_grouped = df[(df['newbie_flag']==1)&(~df['show_dt'].isna())].groupby(['group_id', 'newbie_flag'], as_index=False)[['user_id', 'click_dt', 'approve_dt', 'not_approve_dt', 'order_flag']].agg({'user_id':'count', 'click_dt':'count', 'approve_dt':'count', 'not_approve_dt':'count', 'order_flag':'sum'})

df_grouped['cr_to_approve'] = df_grouped['approve_dt'] / df_grouped['user_id'] * 100
df_grouped['cr_to_ride'] = df_grouped['order_flag'] / df_grouped['user_id'] * 100
df_grouped['cr_to_ride_2'] = df_grouped['order_flag'] / df_grouped['user_id'] * 100
df_grouped['cr_to_not_approve'] = df_grouped['not_approve_dt'] / df_grouped['user_id'] * 100




df_grouped

,group_id,newbie_flag,user_id,click_dt,approve_dt,not_approve_dt,order_flag,cr_to_approve,cr_to_ride,cr_to_ride_2,cr_to_not_approve
0,0,1,2204,2100,1258,204,1065,57.08,48.32,48.32,9.26
1,1,1,2176,2065,1285,221,1070,59.05,49.17,49.17,10.16


In [14]:
df[(df['newbie_flag']==1)&(~df['show_dt'].isna())]

,user_id,group_id,participant_first_toggle_date,registration_date,os_name,city_id,city_name,country_id,country_name,newbie_flag,reg_same_day_flag,filled_flow,registration_dt,show_dt,click_dt,approve_dt,approve_flag,not_approve_dt,not_approve_flag,order_timestamp,order_flag,gmv,rides,orders
2,309434852,1,2025-08-07,2025-08-07,android,4269,Puerto Montt,25,Chile,1,1,liveness,2025-08-07 14:09:16.158000+00:00,2025-08-07 14:12:29.824000+00:00,2025-08-07 14:12:31.174000+00:00,NaT,0,NaT,0,2025-08-07 14:17:31+00:00,1,4.16,1,2
3,309484355,0,2025-08-07,2025-08-07,android,4269,Puerto Montt,25,Chile,1,1,liveness,2025-08-07 20:17:18.344000+00:00,2025-08-07 20:18:22.794000+00:00,2025-08-07 20:18:28.504000+00:00,NaT,0,NaT,0,NaT,0,NaN,<NA>,<NA>
4,309503455,0,2025-08-07,2025-08-07,android,4269,Puerto Montt,25,Chile,1,1,liveness,2025-08-07 23:17:21.297000+00:00,2025-08-07 23:22:10.884000+00:00,2025-08-07 23:22:11.903000+00:00,2025-08-07 23:23:36.714000+00:00,1,NaT,0,2025-08-08 02:14:42+00:00,1,3.20,1,2
6,309710587,1,2025-08-09,2025-08-09,ios,4269,Puerto Montt,25,Chile,1,1,liveness,2025-08-09 03:09:51.181000+00:00,2025-08-09 03:14:33.973000+00:00,2025-08-09 03:14:38.201000+00:00,NaT,0,NaT,0,NaT,0,NaN,<NA>,<NA>
8,309935099,1,2025-08-10,2025-08-10,android,4269,Puerto Montt,25,Chile,1,1,liveness,2025-08-10 05:49:08.103000+00:00,2025-08-10 05:51:38.584000+00:00,2025-08-10 05:51:40.494000+00:00,2025-08-10 05:53:05.213000+00:00,1,NaT,0,2025-08-10 05:53:10+00:00,1,5.39,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2435,313490773,1,2025-08-31,2025-08-31,android,4269,Puerto Montt,25,Chile,1,1,liveness,2025-08-31 04:27:49.562000+00:00,2025-08-31 04:28:10.620000+00:00,2025-08-31 04:28:12.749000+00:00,2025-08-31 04:29:42.057000+00:00,1,NaT,0,2025-08-31 04:29:48+00:00,1,7.24,1,11
2436,314066679,0,2025-09-03,2025-09-03,android,4269,Puerto Montt,25,Chile,1,1,liveness,2025-09-03 15:50:12.033000+00:00,2025-09-03 16:17:02.587000+00:00,2025-09-03 16:17:19.182000+00:00,2025-09-03 16:18:14.492000+00:00,1,NaT,0,2025-09-03 16:18:18+00:00,1,2.14,1,1
2438,314434251,1,2025-09-05,2025-09-05,android,4269,Puerto Montt,25,Chile,1,1,liveness,2025-09-05 23:58:50.210000+00:00,2025-09-06 02:21:35.124000+00:00,2025-09-06 02:21:37.575000+00:00,2025-09-06 02:23:09.277000+00:00,1,NaT,0,2025-09-06 02:23:39+00:00,1,5.18,1,2
2439,314713516,0,2025-09-07,2025-09-07,android,4269,Puerto Montt,25,Chile,1,1,liveness,2025-09-07 14:06:25.181000+00:00,2025-09-07 14:06:59.574000+00:00,2025-09-07 14:07:00.970000+00:00,2025-09-07 14:09:24.908000+00:00,1,NaT,0,2025-09-07 14:09:30+00:00,1,3.11,1,1


# Results

In [2]:
df_res = read_bq("""
WITH newbies AS (SELECT user_id,
                        metric_date
                 FROM indriver-bi.incity.tbl_incity_growth_metrics_detail
                 WHERE user_type = 'pass'
                   AND rides_count > 0
                   AND metric_date >= DATE_ADD(CURRENT_DATE(), INTERVAL -1 YEAR)
                   AND country_id = 25),
     gmv AS (SELECT user_id,
                    SUM(gmv_clean_usd) AS gmv,
                    SUM(rides_count)   AS rides,
                    SUM(orders_count)  AS orders
             FROM indriver-bi.incity.tbl_incity_growth_metrics_detail
             WHERE user_type = 'pass'
               AND metric_date BETWEEN '2025-06-01' AND CURRENT_DATE()
               AND country_id = 25
             GROUP BY 1),
     total AS (SELECT t1.user_id,
                      t1.city_id,
                      geo.city_name,
                      geo.country_id,
                      geo.country_name,
                      IF(group_id = 4546528, 0, 1) AS group_id,
                      participant_first_toggle_date,
                      t2.metric_date,
                      CASE
                          WHEN t2.metric_date IS NULL THEN 1
                          ELSE 0
                          END                         newbie_flag
               FROM indrive-core.ab_platform.tbl_ab_experiment_markup t1
                        JOIN indriver-e6e40.heap.vw_macroregion_mapping geo
                             ON
                                 t1.city_id = geo.city_id
                        LEFT JOIN newbies t2
                                  ON t1.user_id = t2.user_id AND t2.metric_date < t1.participant_first_toggle_date
               WHERE experiment_id = 3713
               QUALIFY ROW_NUMBER() OVER (PARTITION BY t1.user_id ORDER BY metric_date DESC) = 1),
     liveness AS (SELECT user_id,
                         os_name,
                         city_id,
                         city_name,
                         country_id,
                         country_name,
                         COALESCE(filled_flow, 'liveness')                                  AS filled_flow,
                         COALESCE(MAX(IF(name = 'client.verification_start.show', client_time, NULL)),
                                  MAX(IF((name = 'client.verification_flow_result_status.show' AND
                                          LOWER(status) = 'approve'), client_time, NULL)),
                                  MAX(IF((name = 'client.verification_flow_result_status.show' AND
                                          LOWER(status) != 'approve'), client_time, NULL))) AS show_dt,
                         COALESCE(MAX(IF(name = 'client.verification_start.click', client_time, NULL)),
                                  MAX(IF((name = 'client.verification_flow_result_status.show' AND
                                          LOWER(status) = 'approve'), client_time, NULL)),
                                  MAX(IF((name = 'client.verification_flow_result_status.show' AND
                                          LOWER(status) != 'approve'), client_time, NULL))) AS click_dt,
                         MAX(IF((name = 'client.verification_flow_result_status.show' AND
                                 LOWER(status) = 'approve'), client_time, NULL))            AS approve_dt,
                         MAX(IF((name = 'client.verification_flow_result_status.show' AND
                                 LOWER(status) != 'approve'), client_time, NULL))           AS not_approve_dt
                  FROM (SELECT t1.user_id,
                               t1.name,
                               t1.os_name,
                               DATE(TIMESTAMP_MILLIS(t1.client_time))                  AS event_dt_part,
                               TIMESTAMP_MILLIS(t1.client_time)                        AS client_time,
                               t1.city_id,
                               t2.city_name,
                               t2.country_id,
                               t2.country_name,
                               JSON_EXTRACT_SCALAR(payload, '$.verification_flow')     AS verification_flow,
                               IF(JSON_EXTRACT_SCALAR(payload, '$.verification_flow') IS NULL,
                                  LAG(JSON_EXTRACT_SCALAR(payload, '$.verification_flow'))
                                      OVER (PARTITION BY t1.user_id ORDER BY client_time),
                                  JSON_EXTRACT_SCALAR(payload, '$.verification_flow')) AS filled_flow,
                               JSON_EXTRACT_SCALAR(payload, '$.status')                AS status
                        FROM (SELECT *
                              FROM indriver-e6e40.emart.product_event t1
                              WHERE 1 = 1
                                AND name IN (
                                             'client.verification_start.show',
                                             'client.verification_start.click',
                                             'client.verification_flow_result_status.show'
                                  )
                                AND event_dt_part BETWEEN '2025-06-01' AND CURRENT_DATE()
                                AND country_id IN (25)
                              QUALIFY
                                  ROW_NUMBER() OVER (PARTITION BY user_id, name, os_name, event_dt_part, JSON_EXTRACT_SCALAR(payload, '$.verification_flow') ORDER BY client_time DESC) =
                                  1) t1
                                 JOIN indriver-e6e40.heap.vw_macroregion_mapping t2
                                      ON
                                          t1.city_id = t2.city_id
                        WHERE 1 = 1
                          AND name IN (
                                       'client.verification_start.show', 'client.verification_start.click',
                                       'client.verification_flow_result_status.show'
                            ))
                  GROUP BY 1, 2, 3, 4, 5, 6, 7),
     total_liv AS (SELECT t1.user_id,
                          t1.city_id,
                          t1.city_name,
                          t1.country_id,
                          t1.country_name,
                          t1.group_id,
                          t1.participant_first_toggle_date,
                          t1.newbie_flag,
                          t2.user_id AS user_with_svf,
                          t2.os_name,
                          t2.filled_flow,
                          t2.show_dt,
                          t2.click_dt,
                          t2.approve_dt,
                          t2.not_approve_dt
                   FROM total t1
                            JOIN liveness t2
                                 ON t1.user_id = t2.user_id AND DATE(t2.show_dt) >= t1.participant_first_toggle_date),
     rides AS (SELECT order_uuid,
                      user_id    AS pass_id,
                      driver_id,
                      city_id    AS order_city_id,
                      country_id AS order_country_id,
                      status_order,
                      order_timestamp,
                      at_pickup_dttm,
                      departed_pickup_dttm,
                      at_destination_dttm,
                      departed_destination_dttm,
                      driveraccept_timestamp,
                      driverarrived_timestamp,
                      driverstarttheride_timestamp,
                      driverdone_timestamp,
                      clientdone_timestamp,
                      clientcancel_timestamp,
                      drivercancel_timestamp,
                      user_reg_date,
                      driver_reg_date,
                      stage,
                      created_date_order_part,
                      duration_in_seconds
               FROM indriver-e6e40.imart.incity_detail_new_order
               WHERE created_date_order_part BETWEEN '2025-06-01'
                   AND CURRENT_DATE()
                 AND status_order = 'RIDE_STATUS_DONE'
                 AND driveraccept_timestamp IS NOT NULL
                 AND (clientcancel_timestamp IS NULL
                   AND drivercancel_timestamp IS NULL))
SELECT t1.user_id,
       t1.group_id,
       t1.participant_first_toggle_date,
       t1.os_name,
       t1.city_id,
       t1.city_name,
       t1.country_id,
       t1.country_name,
       t1.newbie_flag,
       filled_flow,
       show_dt,
       click_dt,
       approve_dt                                                  AS approve_dt,
       IF(approve_dt IS NOT NULL, 1, 0)                            AS approve_flag,
       not_approve_dt,
       IF(not_approve_dt IS NOT NULL AND approve_dt IS NULL, 1, 0) AS not_approve_flag,
       t2.order_timestamp,
       IF(t2.order_timestamp IS NOT NULL, 1, 0)                    AS order_flag,
       t3.gmv,
       t3.rides,
       t3.orders
FROM total_liv t1
         LEFT JOIN rides t2
                   ON t1.user_id = t2.pass_id AND
                      t2.created_date_order_part >= participant_first_toggle_date
         LEFT JOIN gmv t3 ON t1.user_id = t3.user_id
QUALIFY ROW_NUMBER() OVER (PARTITION BY t1.user_id ORDER BY t2.order_timestamp) = 1
""")

df_res.head()

,user_id,group_id,participant_first_toggle_date,os_name,city_id,city_name,country_id,country_name,newbie_flag,filled_flow,show_dt,click_dt,approve_dt,approve_flag,not_approve_dt,not_approve_flag,order_timestamp,order_flag,gmv,rides,orders
0,21921294,0,2025-08-07,android,4269,Puerto Montt,25,Chile,0,liveness,2025-09-06 22:17:33.475000+00:00,2025-09-06 22:17:34.708000+00:00,2025-09-06 22:18:35.560000+00:00,1,NaT,0,2025-08-07 03:46:05+00:00,1,92.45,38,61
1,62136015,1,2025-08-07,android,4269,Puerto Montt,25,Chile,0,liveness,2025-09-06 02:04:06.580000+00:00,2025-09-06 02:04:13.094000+00:00,2025-09-06 02:04:43.664000+00:00,1,NaT,0,2025-08-07 02:29:09+00:00,1,132.62,45,75
2,75149470,0,2025-08-18,ios,4269,Puerto Montt,25,Chile,0,liveness,2025-08-18 19:24:53.882000+00:00,2025-08-18 19:24:54.925000+00:00,2025-08-18 19:25:28.427000+00:00,1,NaT,0,2025-08-20 03:54:37+00:00,1,144.31,50,57
3,118829587,1,2025-08-12,android,4269,Puerto Montt,25,Chile,0,documents,2025-08-12 01:34:40.320000+00:00,2025-08-12 01:34:46.376000+00:00,2025-08-12 01:35:32.171000+00:00,1,NaT,0,2025-08-12 01:35:37+00:00,1,69.83,24,49
4,166519297,1,2025-08-07,android,4269,Puerto Montt,25,Chile,0,liveness,2025-08-29 19:55:10.758000+00:00,2025-08-29 19:55:19.852000+00:00,2025-08-29 19:55:52.289000+00:00,1,NaT,0,2025-08-11 10:53:20+00:00,1,78.07,21,31


In [3]:
df_res['rides'] = df_res['rides'].fillna(0)
df_res['gmv'] = df_res['gmv'].fillna(0)
df_res.head()

def method_benjamini_hochberg(
    pvalues: np.ndarray,
    alpha: float = 0.05
    ) -> np.ndarray:
    """Apply the Benjamini-Hochberg procedure for multiple hypothesis testing."""
    m = len(pvalues)
    array_alpha = np.arange(1, m + 1) * alpha / m
    sorted_pvalue_indexes = np.argsort(pvalues)
    res = np.zeros(m)
    for idx, pvalue_index in enumerate(sorted_pvalue_indexes):
        pvalue = pvalues[pvalue_index]
        alpha_ = array_alpha[idx]
        if pvalue <= alpha_:
            res[pvalue_index] = 1
        else:
            break
    return res.astype(int)

In [13]:
df_res[(df_res['newbie_flag']==1)&(~df_res['show_dt'].isna())][['user_id', 'group_id', 'os_name', 'approve_flag', 'not_approve_flag', 'order_flag', 'rides', 'gmv']]

,user_id,group_id,os_name,approve_flag,not_approve_flag,order_flag,rides,gmv
7,70243158,1,ios,1,0,1,23,117.08
72,309143426,1,android,1,0,1,31,95.99
82,206674700,1,android,1,0,1,37,107.34
84,309463419,0,ios,1,0,1,30,137.68
98,290490329,1,ios,1,0,1,23,114.11
...,...,...,...,...,...,...,...,...
7503,22007717,0,ios,1,0,1,20,60.17
7516,311416000,0,android,1,0,1,20,83.44
7519,310932014,0,ios,1,0,1,20,69.81
7550,309877845,1,android,1,0,1,22,49.15


In [4]:
indicators = ['approve_flag', 'not_approve_flag', 'order_flag', 'rides', 'gmv']
res_df = pd.DataFrame()

for metric in indicators:

    if metric in ['rides', 'gmv']:
        ttest = expab.ttest(
            df_res[(df_res['newbie_flag']==1)&(~df_res['show_dt'].isna())][['user_id', 'group_id', 'os_name', 'approve_flag', 'not_approve_flag', 'order_flag', 'rides', 'gmv']],
            metric,
            'group_id'
            )
        ttest['criteria'] = 'ttest'
        res_df = pd.concat([res_df, ttest])

    else:
        ztest = expab.ztest_proportion(
            df_res[(df_res['newbie_flag']==1)&(~df_res['show_dt'].isna())][['user_id', 'group_id', 'os_name', 'approve_flag', 'not_approve_flag', 'order_flag', 'rides', 'gmv']],
            metric,
            'group_id'
            )
        ztest['criteria'] = 'ztest'
        res_df = pd.concat([res_df, ztest])

    res_df['significance'] = (res_df['pvalue']<0.05)*1
    res_df['result_with_corr'] = method_benjamini_hochberg(res_df['pvalue'].values)

res_df

,metric_name,group0_sample_size,group1_sample_size,group0,group1,statistic,pvalue,mean0,mean1,diff_mean,diff_mean_%,lb,ub,lb_%,ub_%,criteria,significance,result_with_corr
0,approve_flag,2942,2946,0,1,-0.55,0.58,0.59,0.59,0.01,1.19,-0.02,0.03,-3.08,5.47,ztest,0,0
0,not_approve_flag,2942,2946,0,1,-0.79,0.43,0.09,0.10,0.01,6.55,-0.01,0.02,-9.79,22.88,ztest,0,0
0,order_flag,2942,2946,0,1,-0.23,0.81,0.50,0.50,0.00,0.61,-0.02,0.03,-4.50,5.72,ztest,0,0
0,rides,2942,2946,0,1,0.17,0.86,1.95,1.93,-0.02,-1.08,-0.26,0.22,-13.22,11.07,ttest,0,0
0,gmv,2942,2946,0,1,0.06,0.95,7.54,7.51,-0.03,-0.38,-0.97,0.91,-12.85,12.09,ttest,0,0
